### Setup

In [4]:
import pandas as pd
import ast
import os

In [5]:
local = pd.read_csv('datasets/crawler/territory_codes_local.csv')
foreign = pd.read_csv('datasets/crawler/territory_codes_foreign.csv')
codes = (
    pd.concat([local, foreign])
    .rename(columns={'name': 'freguesia'})
    .reset_index(drop=True)
)

results = pd.DataFrame()
metadata = pd.DataFrame()

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/crawler/territory_codes_local.csv'

### Methods

In [3]:
# Returns resuts table df
def getResultsTable(file):
    resultsParty = file.loc['resultsParty', 'currentResults']

    results = (
        pd.DataFrame(ast.literal_eval(resultsParty))
        .drop(['absoluteMajority', 'constituenctyCounter', 'imageKey', 'mandates', 'presidents'], axis=1)
        .rename(columns={'acronym': 'party'})
        .set_index('party')
    )

    return results

# Returns metadata df
def getMetadata(file, row):
    cr = pd.Series(
        file['currentResults']
        .drop(['availableMandates', 'compensation', 'displayMessage', 'hasNoVoting', 'resultsParty'
            , 'tie', 'tieMessage', 'totalBoycotts', 'totalForeignBoycotts', 'totalLocalBoycotts'
            , 'totalMandates', 'totalParishesApproved'])
    )

    metadata = (
        file.drop(['currentResults', 'displayMessage', 'foreignCounterMessage', 'nationalCounterMessage', 'previousResults'
                    , 'refreshTimeout', 'territoryMessage', 'time', 'timeOnServer', 'warningMessage'], axis=1)
        .loc['availableMandates', :]
    )
    
    metadata['district'] = row['distrito']
    metadata[''] = row['concelho']

    metadata = (
        pd.concat([metadata, cr])
        .to_frame(name='metadata')
        .T
        .set_index('territoryKey')
    )

    return metadata

# Returns results df
def getResults(file):
    r = getResultsTable(file)
    metadata = getMetadata(file)
    r['tkey'] = metadata.index[0]
    r['parish'] = metadata['territoryFullName'].iloc[0]

# ! Check if you should reset and then set
    r = r.reset_index().set_index(['tkey', 'party'])

    return r

# Creates directories
def mkdir():
        path = '/datasets/Results/' 

        working_dir = os.getcwd()
        newpath = working_dir + path

        if not os.path.exists(newpath):
            os.makedirs(newpath)

SyntaxError: invalid syntax (3801322431.py, line 29)

### Create Results files

In [ ]:
for i, row in codes.iterrows():
    loc = row['territoryKey'].split('-')[0].capitalize()
    file = (
        pd.read_csv('datasets/' + loc + '/' + row['distrito'] + '/' + row['concelho'] + '/' + row['freguesia'] + '.csv')
        .set_index('index')
    )

    results = pd.concat([results, getResults(file)])
    metadata = pd.concat([metadata, getMetadata(file, row)])

TypeError: getMetadata() missing 1 required positional argument: 'row'

In [ ]:
results

percentage  validVotesPercentage  votes  \
tkey           party                                                         
LOCAL-430101   PPD/PSD.CDS-PP.PPM       43.17                 44.72    199   
               PS                       34.27                 35.51    158   
               CH                       12.15                 12.58     56   
               B.E.                      2.17                  2.25     10   
               IL                        1.95                  2.02      9   
...                                       ...                   ...    ...   
FOREIGN-939999 JPP                       0.22                  0.33     13   
               MPT.A                     0.17                  0.26     10   
               ND                        0.13                  0.21      8   
               R.I.R.                    0.12                  0.18      7   
               E                         0.08                  0.13      5   

                                                                              parish  
tkey           party                                                                  
LOCAL-430101   PPD/PSD.CDS-PP.PPM                                            Altares  
               PS                                                            Altares  
               CH                                                            Altares  
               B.E.                                                          Altares  
               IL                                                            Altares  
...                                                                              ...  
FOREIGN-939999 JPP                 Restantes Postos Consulares Dos Países da Ásia...  
               MPT.A               Restantes Postos Consulares Dos Países da Ásia...  
               ND                  Restantes Postos Consulares Dos Países da Ásia...  
               R.I.R.              Restantes Postos Consulares Dos Países da Ásia...  
               E                   Restantes Postos Consulares Dos Países da Ásia...  

[45736 rows x 4 columns]

In [ ]:
mkdir()

results.to_csv('datasets/Results/results.csv')
metadata.to_csv('datasets/Results/metadata.csv')

### Explore Data

In [ ]:
results = pd.read_csv('datasets/Results/results.csv')
metadata = pd.read_csv('datasets/Results/metadata.csv')

parties = results['party'].unique()
results = results.set_index(['parish', 'party'])

idx = pd.IndexSlice

In [ ]:
results.loc['Angra (Sé)', 'PS']

C:\Users\druid\AppData\Local\Temp\ipykernel_5312\3641195549.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  results.loc['Angra (Sé)', 'PS']


,,tkey,percentage,validVotesPercentage,votes
parish,party,,,,
Angra (Sé),PS,LOCAL-430105,24.88,25.54,153


In [ ]:
# Results by Party
# for i, party in parties:
#     results.loc[idx[:, party], idx[:]]

## Database

In [ ]:
# results
# metadata


In [ ]:
# Send to SQL
import mysql.connector
from dotenv import load_dotenv

load_dotenv()
user = os.getenv('USER')
password = os.getenv('PW')

cnx = mysql.connector.connect(
    user=user,
    password=password,
    host="localhost",
    database="legislativas"
)


